# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Parts Model' {
        package 'Fake Library' {
            part def Part;
            port def Port;
            connection def Connection;
        }
    
        part 'Power Group' : 'Fake Library'::Part {
            part 'Power Source' : 'Fake Library'::Part {
                port 'Power Out' : 'Fake Library'::Port;
            }
            part 'Power User' : 'Fake Library'::Part {
                port 'Power In' : 'Fake Library'::Port;
            }

            connection powerToUser : 'Fake Library'::Connection connect 'Power Source'::'Power Out' to 'Power User'::'Power In';
        }
    
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import os
from pathlib import Path
import networkx as nx

import pymbe.api as pm

from pymbe.client import SysML2Client

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

from pymbe.interpretation.calc_dependencies import generate_execution_order
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.label import get_label

from pymbe.query.query import roll_up_multiplicity
from pymbe.query.metamodel_navigator import map_inputs_to_results

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [2]:
helper_client = SysML2Client()

simple_parts_file = Path("..") / "tests/fixtures/Simple Parts Model.json"

helper_client._load_from_file(simple_parts_file)

Create a graph representation of the model and load it into memory.

In [3]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [4]:
shorten_pre_bake = {
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [5]:
part_id = 'a4b4fc39-eec4-4ae6-8f16-62a106323e69'
port_id = 'ef2b63dc-3666-4df0-beb9-790b8e7fc21c'
connection_id = '290107ad-b043-405e-8a60-2796e7681552'

power_in_id = '6717616c-47ee-4fed-bf7d-e4e98c929fac'
power_user_id = '3f314dcb-4426-48ae-a21e-cac3dbf9deff'
connect_use_id = '8f3cfffc-04c8-458b-84d4-82ef1bf2498c'

In [6]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [7]:
get_types_for_feature(lpg, 'f91dbe6e-ca1b-48fd-9b7d-b0a5325d0e05')

[]

In [8]:
get_features_typed_by_type(lpg, part_id)

['009a03de-7718-47c4-99c1-5c80234536bf',
 '3f314dcb-4426-48ae-a21e-cac3dbf9deff',
 '07d69b3a-1340-4b21-8aac-135f822a86bd']

In [9]:
roll_up_multiplicity_for_type(lpg, helper_client.elements_by_id[connection_id], 'upper')

c:\sandboxes\pymbe\src\pymbe\graph\lpg.py:400: UserWarning: These edge types are not in the graph: {'FeatureValue', 'ParameterMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


4

In [10]:
roll_up_multiplicity(lpg, helper_client.elements_by_id[power_in_id], 'upper')

4

In [11]:
roll_up_multiplicity(lpg, helper_client.elements_by_id[power_user_id], 'upper')

4

In [12]:
roll_up_multiplicity(lpg, helper_client.elements_by_id[connect_use_id], 'upper')

4

In [13]:
banded_featuring_graph = lpg.get_projection("Expanded Banded Graph")

In [14]:
banded_roots = [
        node
        for node in banded_featuring_graph.nodes
        if banded_featuring_graph.out_degree(node) < 1
    ]
banded_roots

['009a03de-7718-47c4-99c1-5c80234536bf',
 '004d6e92-6abc-4d12-8879-cc4b951f8119',
 'a67b9fc3-8e5d-4c0e-9775-45fee7b014ec',
 '9500953f-d528-44a6-b5fa-8b2f2933c63d']

In [15]:
from pymbe.interpretation.interp_playbooks import build_expression_sequence_templates
from pymbe.label import get_label_for_id

In [16]:
expr_seqs = build_expression_sequence_templates(lpg)
for seq in expr_seqs:
    sig_seq = []
    for item in seq:
        sig_seq.append(get_label_for_id(item, helper_client.elements_by_id))
    print(sig_seq)

['4 «Occurred LiteralInteger»', '$result']
['4 «Occurred LiteralInteger»', '$result']
['2 «Occurred LiteralInteger»', '$result']


In [17]:
from pymbe.query.metamodel_navigator import feature_multiplicity

In [18]:
feature_multiplicity(helper_client.elements_by_id[connection_id], helper_client.elements_by_id, "upper")

1

In [19]:
from pymbe.interpretation.interp_playbooks import build_sequence_templates
build_sequence_templates(lpg)

[['009a03de-7718-47c4-99c1-5c80234536bf',
  '8f3cfffc-04c8-458b-84d4-82ef1bf2498c'],
 ['009a03de-7718-47c4-99c1-5c80234536bf',
  '3f314dcb-4426-48ae-a21e-cac3dbf9deff',
  '6717616c-47ee-4fed-bf7d-e4e98c929fac'],
 ['009a03de-7718-47c4-99c1-5c80234536bf',
  '07d69b3a-1340-4b21-8aac-135f822a86bd',
  '4cd714eb-796a-44b4-8864-daf18bd04f4a']]

In [20]:
m0_interpretation = random_generator_playbook(
    lpg,
    shorten_pre_bake
)

[0, 1, 1, 0]
[0, 3, 1, 2]


c:\sandboxes\pymbe\src\pymbe\graph\lpg.py:400: UserWarning: These edge types are not in the graph: {'ResultExpressionMembership', 'ParameterMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\sandboxes\pymbe\src\pymbe\graph\lpg.py:400: UserWarning: These edge types are not in the graph: {'Superclassing'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [21]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

['Power Group: Part', 'powerToUser: Connection'], 4
['Power Group: Part', 'Power User: Part', 'Power In: Port'], 4
['Power Group: Part', 'Power Source: Part', 'Power Out: Port'], 2


## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

Show all interpretation sequence sets (limited to length of 5).

In [22]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)

Port, id = ef2b63dc-3666-4df0-beb9-790b8e7fc21c, size = 6
[Port#0]
[Port#1]
[Port#2]
[Port#3]
[Port#4]
['..']
Part, id = a4b4fc39-eec4-4ae6-8f16-62a106323e69, size = 7
[Part#0]
[Part#1]
[Part#2]
[Part#3]
[Part#4]
['..']
Connection, id = 290107ad-b043-405e-8a60-2796e7681552, size = 4
[Connection#0]
[Connection#1]
[Connection#2]
[Connection#3]
Power Group: Part, id = 009a03de-7718-47c4-99c1-5c80234536bf, size = 1
[Part#0]
powerToUser: Connection, id = 8f3cfffc-04c8-458b-84d4-82ef1bf2498c, size = 4
[Part#0, Connection#3]
[Part#0, Connection#0]
[Part#0, Connection#2]
[Part#0, Connection#1]
Power User: Part, id = 3f314dcb-4426-48ae-a21e-cac3dbf9deff, size = 4
[Part#0, Part#3]
[Part#0, Part#4]
[Part#0, Part#5]
[Part#0, Part#6]
Power In: Port, id = 6717616c-47ee-4fed-bf7d-e4e98c929fac, size = 4
[Part#0, Part#3, Port#1]
[Part#0, Part#4, Port#2]
[Part#0, Part#5, Port#0]
[Part#0, Part#6, Port#5]
Power Source: Part, id = 07d69b3a-1340-4b21-8aac-135f822a86bd, size = 2
[Part#0, Part#6]
[Part#0, Par